In [1]:
#### **CARTPOLE DQN V1 (Plain)**

This is a naive version of the CARTPOLE DQN algorithm <br>
It will be very slow but it is an educational tool to understand all the steps in the DQN algorithm <br>


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import random
import gymnasium as gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [3]:
EPISODES = 1000

memory = deque(maxlen=2000)
gamma = 0.95    # discount rate
epsilon = 1.0  # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001

verb = 0   # to see traces

# Environment setup
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [4]:
def build_model(state_size,action_size):
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse',
                      optimizer=Adam(learning_rate=learning_rate))
    model.summary()
    return model

In [5]:
def store(state, action, reward, next_state, done):             # memorize
    memory.append((state, action, reward, next_state, done))

def select_action_greedy(state, DQN):                                      # act
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    act_values = DQN.predict(state, verbose=verb)
    return np.argmax(act_values[0])  # returns action selected with greedy strategy

def replay(batch_size, DQN, epsilon):
        minibatch = random.sample(memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + gamma * np.amax(DQN.predict(next_state, verbose=verb)[0]))
            target_f = DQN.predict(state, verbose=verb)
            target_f[0][action] = target
            DQN.fit(state, target_f, epochs=1, verbose=0)
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay
        return epsilon

def load(name, DQN):
    DQN.load_weights(name)

def save(name, DQN):
    DQN.save_weights(name)

In [ ]:
rewards_per_episode= []
DQN = build_model(state_size, action_size)

env = gym.make('CartPole-v1')

    # agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])

    for time in range(500):
            
        action = select_action_greedy(state, DQN)
        next_state, reward, done, truncated , _ = env.step(action)
        if truncated:
                done = True
        reward = reward if not done else -10                      
        next_state = np.reshape(next_state, [1, state_size])
        store(state, action, reward, next_state, done)
        state = next_state
        rewards_per_episode.append(reward)
        if done:
            rw_ep = 0
            if e > 100:
                rw_ep = np.mean(rewards_per_episode[-100:])
            print("episode: {}/{}, score: {}, e: {:.2}, rw_100: {:-2}"
                      .format(e, EPISODES, time, epsilon, rw_ep))

            break
        if len(memory) > batch_size:
            epsilon = replay(batch_size, DQN, epsilon)

/home/jau/DL-Keras/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732960234.959091 2427947 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732960234.976629 2427947 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732960234.976665 2427947 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732960234.979238 2427947 cuda_executor.cc:1001] could 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24)             │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            50 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 770 (3.01 KB)

 Trainable params: 770 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

episode: 0/1000, score: 13, e: 1.0, rw_100:  0
episode: 1/1000, score: 17, e: 1.0, rw_100:  0


I0000 00:00:1732960236.383207 2428025 service.cc:146] XLA service 0x7fb424017410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732960236.383250 2428025 service.cc:154]   StreamExecutor device (0): NVIDIA T600 Laptop GPU, Compute Capability 7.5
I0000 00:00:1732960237.311610 2428025 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


episode: 2/1000, score: 25, e: 0.88, rw_100:  0
episode: 3/1000, score: 19, e: 0.8, rw_100:  0
episode: 4/1000, score: 20, e: 0.73, rw_100:  0
episode: 5/1000, score: 9, e: 0.69, rw_100:  0
episode: 6/1000, score: 13, e: 0.65, rw_100:  0
episode: 7/1000, score: 13, e: 0.61, rw_100:  0
episode: 8/1000, score: 12, e: 0.57, rw_100:  0
episode: 9/1000, score: 10, e: 0.55, rw_100:  0
episode: 10/1000, score: 11, e: 0.52, rw_100:  0
episode: 11/1000, score: 14, e: 0.48, rw_100:  0
episode: 12/1000, score: 29, e: 0.42, rw_100:  0
episode: 13/1000, score: 13, e: 0.39, rw_100:  0
episode: 14/1000, score: 10, e: 0.37, rw_100:  0
episode: 15/1000, score: 12, e: 0.35, rw_100:  0
episode: 16/1000, score: 12, e: 0.33, rw_100:  0
episode: 17/1000, score: 19, e: 0.3, rw_100:  0
episode: 18/1000, score: 15, e: 0.28, rw_100:  0
episode: 19/1000, score: 21, e: 0.25, rw_100:  0
episode: 20/1000, score: 37, e: 0.21, rw_100:  0
episode: 21/1000, score: 27, e: 0.18, rw_100:  0
episode: 22/1000, score: 30, e: